# Runnables

## Métodos dos Runnables de LCEL

A interface padrão de LCEL inclui os seguintes métodos:

- **stream:** transmitir de volta fragmentos da resposta
- **invoke:** chamar a cadeia com um input
- **batch:** chamar a cadeia com uma lista de inputs

Esses também possuem métodos assíncronos correspondentes que devem ser usados com a sintaxe `asyncio await` para concorrência:

- **astream:** transmitir de volta fragmentos da resposta de forma assíncrona
- **ainvoke:** chamar a cadeia com um input de forma assíncrona
- **abatch:** chamar a cadeia com uma lista de inputs de forma assíncrona

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv, find_dotenv
from langchain_core.runnables import RunnableLambda, RunnableParallel
import os

In [2]:
#! TODO - Instantiate your chat model
# * Carrega as variáveis de ambiente
_ = load_dotenv(find_dotenv())

# * Verifica se a API key está configurada
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# * Configura o modelo com parâmetros específicos
model: ChatOpenAI = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.0,  # Controla a criatividade das respostas
)  # type: ignore

response = model.invoke("The Sky is ?")
print(response.content)

The sky is **blue** (during the day) due to the scattering of sunlight by the atmosphere. This phenomenon is called **Rayleigh scattering**. At sunrise and sunset, the sky can appear **red, orange, or pink** because the sunlight passes through more atmosphere, scattering shorter blue wavelengths and allowing longer red wavelengths to dominate.

At night, the sky appears **dark or black**, dotted with stars, because the sun is no longer illuminating the atmosphere above you.

So, the sky **can be blue, red, orange, pink, gray, or black** depending on the time of day, weather, and atmospheric conditions!


In [3]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")

chain = prompt | model

## Invoke

O invoke é o método básico para inserir uma input na cadeia e receber uma resposta

In [4]:
chain.invoke({'assunto': 'cachorrinhos'})

AIMessage(content='Os cachorrinhos são seres incrivelmente leais e amorosos, capazes de nos trazer alegria e companhia em todos os momentos da vida.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 20, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C1Alc1SagKFG65a3UqzkJpFnfFcTp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--306fe8e9-40fd-427a-9810-bbd6db36ed64-0', usage_metadata={'input_tokens': 20, 'output_tokens': 37, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Ele pode ser rodado como uma simples string quando existe apenas uma input no prompt, mas a forma mais recomendada é informando especificamente o nome da input através de um dicionário.

In [5]:
chain.invoke('cachorrinhos')

AIMessage(content='Cachorrinhos são anjos de quatro patas que enchem nossas vidas de amor e alegria.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 20, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C1AlrE2b9GWv0Q4hknIQjmcBsk9ir', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4bb0ae23-4f5b-4c22-a761-2f369d47456b-0', usage_metadata={'input_tokens': 20, 'output_tokens': 27, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Stream

Para recebermos uma saída conforme ela é gerada pelo modelo utilizamos o stream


In [6]:
for stream in chain.stream('cachorrinhos'):
    print(stream.content, end='') 

Cachorrinhos nos ensinam a amar incondicionalmente e nos alegram todos os dias com sua alegria contagiante.

## Batch

Para fazermos múltiplas requisições em paralelo utilizamos o batch

In [7]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinhos'}, {'assunto': 'cavalinhos'}])

[AIMessage(content='Cachorrinhos são a pura essência de amor e fofura, capazes de alegrar nossos dias com apenas um olhar.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 20, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C1AyWymdUbi4Zd1zwjXookBoukZFM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--03ff903a-2f7a-4b8c-a255-3d61e02780f4-0', usage_metadata={'input_tokens': 20, 'output_tokens': 33, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='"Os gatinhos são verdadeiras bolinhas de pelo fofinhas e cheias de amor para dar."', additi

In [8]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinhos'}, {'assunto': 'cavalinhos'}], config={'max_concurrency': 2})

[AIMessage(content='Cachorrinhos são seres repletos de amor e lealdade, que nos ensinam o verdadeiro significado da felicidade e companheirismo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 20, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C1AygZKvl7cXykouJrBd8eJpwJLZn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--17817e92-b9cf-41c7-a668-5a0a0f7c3ad5-0', usage_metadata={'input_tokens': 20, 'output_tokens': 38, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='Os gatinhos são seres adoráveis e fofos que trazem alegria e carinho 

## Runnables especiais

### Rodando em paralelo
```
     Input      
      / \       
     /   \      
 Branch1 Branch2
     \   /      
      \ /       
      Combine   
```

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel


model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Crie um nome para o seguinte produto: {produto}")

chain_nome = prompt | model | StrOutputParser()

In [10]:
from langchain_core.runnables import RunnableLambda

logs = []
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Descreva o cliente potencial para o seguinte produto: {produto}")

parser = StrOutputParser()
parser_and_log_output_chain = RunnableParallel(
    output_parser=parser,
    log = RunnableLambda(lambda x: logs.append(x))
)

chain_clientes = prompt | model | parser_and_log_output_chain

response = chain_clientes.invoke({"produto": "notebook"})
print(response)
print(logs)

{'output_parser': 'O cliente potencial para um notebook pode ser bastante variado, já que se trata de um produto de uso muito comum nos dias de hoje. No entanto, podemos identificar algumas características comuns entre os potenciais compradores desse tipo de produto.\n\nEm primeiro lugar, o público-alvo de notebooks costuma ser formado por pessoas que trabalham em escritórios ou que precisam ter acesso constante à internet e a ferramentas de trabalho, como estudantes, profissionais liberais, empreendedores e executivos.\n\nAlém disso, o cliente potencial para um notebook geralmente possui um estilo de vida bastante dinâmico, precisa de mobilidade e praticidade no dia a dia e valoriza a tecnologia e a inovação em seus dispositivos ele eletrônicos.\n\nPor fim, é importante ressaltar que o público interessado em notebooks pode ter diferentes faixas etárias e níveis de conhecimento em tecnologia, sendo necessário oferecer produtos de diferentes faixas de preço e especificações técnicas par

In [11]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("""Dado o produto com o seguinte nome e seguinte
público potencial, desenvolva um anúncio para o produto.
                                          
Nome do produto: {nome_produto}
Público: {publico}""")

In [12]:
parallel = RunnableParallel({'nome_produto': chain_nome, 'publico': chain_clientes})
parallel.invoke({'produto': 'Um copo inquebrável'})

{'nome_produto': 'Ultra Resist Cup',
 'publico': {'output_parser': 'O cliente potencial para um copo inquebrável seria alguém que valoriza a durabilidade e resistência de seus utensílios domésticos. Pode ser uma pessoa que tem crianças pequenas em casa, que são propensas a derrubar e quebrar objetos acidentalmente, ou alguém que gosta de praticar atividades ao ar livre e precisa de um copo resistente para levar em suas aventuras. Além disso, pessoas que trabalham em ambientes mais agitados ou movimentados também podem se interessar por um copo inquebrável, pois não terão que se preocupar com possíveis danos causados por quedas acidentais.',
  'log': None}}

In [13]:
chain = parallel | prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'produto': 'Um copo inquebrável'})

'🌟 Apresentamos o Copo Ultrarresistente, o companheiro ideal para quem vive uma vida agitada e está sempre em movimento! 🌟\n\nSe você é um pai ou mãe de crianças pequenas, um esportista, um festeiro ou alguém que simplesmente tem o azar de derrubar coisas com frequência, o Copo Ultrarresistente foi feito pensando em você!\n\nFeito com um material super resistente, esse copo é praticamente inquebrável, garantindo que você possa usá-lo sem preocupações em qualquer situação do dia a dia. Seja no trabalho, em casa ou durante suas atividades favoritas, o Copo Ultrarresistente estará lá para te acompanhar.\n\nNão deixe mais seus utensílios domésticos quebrarem facilmente! Invista em durabilidade e praticidade com o Copo Ultrarresistente. Peça já o seu! 🌟🥤🔝'

In [14]:
from langchain_core.runnables import RunnableLambda

def cumprimentar(nome):
    return f'Olá, {nome}!'

runnable_cumprimentar = RunnableLambda(cumprimentar)

resultado = runnable_cumprimentar.invoke('Maria')
print(resultado)

Olá, Maria!


In [15]:
from langchain_core.runnables import RunnablePassthrough
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("""Dado o produto com o seguinte nome e seguinte
público potencial, desenvolva um anúncio para o produto.
                                          
Nome do produto: {nome_produto}
Característica do produto: {produto}
Público: {publico}""")

parallel = RunnablePassthrough().assign(**{'nome_produto': chain_nome, 'publico': chain_clientes})
chain = parallel | prompt | ChatOpenAI() | StrOutputParser()
chain.invoke({'produto': 'Copo inquebrável'})

'🌟 Anúncio 🌟\n\nProcurando por um copo resistente e durável que não quebra facilmente? Conheça o DurávelCopo StrongFlex!\n\nFeito com material de alta qualidade, nosso copo inquebrável é perfeito para uso diário em ambientes agitados, como casas com crianças e animais de estimação. Além disso, é ideal para atividades ao ar livre, como acampamentos, festas e eventos, pois resistente a impactos e quedas.\n\nNão se preocupe mais com copos quebrados! Garanta já o seu DurávelCopo StrongFlex e tenha um produto de qualidade e durabilidade incomparáveis.\n\nAproveite essa oportunidade e adquira o seu copo inquebrável agora mesmo! 🌟🥤👌 #DurávelCopo #StrongFlex #Durabilidade #Resistência #CopoInquebrável'